![](https://i.imgur.com/eBRPvWB.png)

# Generowanie poezji za pomocą RNN i PyTorch w oparciu o sylaby.

[W tutorialu](https://github.com/spro/practical-pytorch/blob/master/char-rnn-classification/char-rnn-classification.ipynb) użyliśmy RNN, aby sklasyfikować tekst po jednym znaku na raz. Tym razem wygenerujemy tekst po jedej sylabie na raz.
```
> python generate.py -n 500

PAOLTREDN:
Let, yil exter shis owrach we so sain, fleas,
Be wast the shall deas, puty sonse my sheete.

BAUFIO:
Sirh carrow out with the knonuot my comest sifard queences
O all a man unterd.

PROMENSJO:
Ay, I to Heron, I sack, againous; bepear, Butch,
An as shalp will of that seal think.

NUKINUS:
And house it to thee word off hee:
And thou charrota the son hange of that shall denthand
For the say hor you are of I folles muth me?
```

This one might make you question the series title — "is that really practical?" However, these sorts of generative models form the basis of machine translation, image captioning, question answering and more.

Zobacz [Sequence to Sequence Translation tutorial](https://github.com/spro/practical-pytorch/blob/master/seq2seq-translation/seq2seq-translation.ipynb) żeby nauczyć się więcej w tym temacie.

## Warto przeczytać

Zakładam, że zainstalowałeś przynajmniej PyTorch, znasz Pythona i rozumiesz Tensory:

* http://pytorch.org/ Instrukcje instalacji
* [Deep Learning with PyTorch: A 60-minute Blitz](https://github.com/pytorch/tutorials/blob/master/Deep%20Learning%20with%20PyTorch.ipynb) jak zacząć z PyTorch (ogólnie)
* [jcjohnson's PyTorch examples](https://github.com/jcjohnson/pytorch-examples) dla szczegółowego przeglądu
* [Introduction to PyTorch for former Torchies](https://github.com/pytorch/tutorials/blob/master/Introduction%20to%20PyTorch%20for%20former%20Torchies.ipynb) jeśli byłeś użytkownikiem Lua Torch

Przydałoby się również wiedzieć o RNN'ach i ich działaniu:

* [The Unreasonable Effectiveness of Recurrent Neural Networks](http://karpathy.github.io/2015/05/21/rnn-effectiveness/) pokazuje kilka przykładów z życia
* [Understanding LSTM Networks](http://colah.github.io/posts/2015-08-Understanding-LSTMs/) dotyczy LSTM'ów, ale też podaje ogólne informacje na temat RNN

Zobacz także podobne tutoriale z serii:

* [Classifying Names with a Character-Level RNN](https://github.com/spro/practical-pytorch/blob/master/char-rnn-classification/char-rnn-classification.ipynb) używa RNN do klasyfikacji
* [Generating Names with a Conditional Character-Level RNN](https://github.com/spro/practical-pytorch/blob/master/conditional-char-rnn/conditional-char-rnn.ipynb) opierając się na tym modelu, dodaje kategorię jako dane wejściowe

## Przygotowanie danych

Plik wejściowy to duży plik tekstowy. Dzielimy go na sylaby.

### Podzielenie korpusu na sylaby

In [ ]:
from pathlib import Path
dataset_path = Path('data/rnn_generator'); dataset_path
tmp_path = dataset_path / 'tmp/'
!mkdir -p $tmp_path

In [ ]:
ls -lah $dataset_path/

In [ ]:
fn_corpus_char = dataset_path/'pan_tadeusz.txt'
fn_corpus_syl = dataset_path/'pan_tadeusz.syl1.txt'

In [ ]:
platform_suffixes = {'Linux': 'linux', 'Darwin': 'macos'}
import platform
platform_suffix = platform_suffixes[platform.system()]
stemmer_bin = f'bin/stemmer.{platform_suffix}'

In [ ]:
# !$stemmer_bin -h

In [ ]:
!$stemmer_bin -s 7683 -v -d bin/stemmer2.dic -i $fn_corpus_char -o $fn_corpus_syl

### Załadowanie danych

Ładujemy plik do pamięci i tokenizujemy. Tworzymy też listę wszystkich tokenów `all_tokens`.

In [ ]:
import string
import random
import re

file = open(fn_corpus_syl).read()
file_len = len(file)
print('file_len =', file_len)

In [ ]:
# taken from fastai/text.py
import re, string
# remove +,- chars from punctuation set to keep syllables e.g.'--PO++' intact
punctuation=re.sub('[\+-]', '', string.punctuation)
re_tok = re.compile(f'([{punctuation}“”¨«»®´·º½¾¿¡§£₤‘’])')
def tokenize(s): return re_tok.sub(r' \1 ', s).split()

file_tok = tokenize(file); len(file_tok), file_tok[:8]
file_tok_len = len(file_tok)

all_tokens = sorted(list(set(file_tok)))
n_tokens = len(all_tokens); print(n_tokens, all_tokens[:50])

Aby stworzyć 'wejścia' z tego dużego ciągu danych, podzielimy go na kawałki.

In [ ]:
chunk_len = 400

def random_chunk():
    start_index = random.randint(0, file_tok_len - chunk_len)
    end_index = start_index + chunk_len + 1
    return file_tok[start_index:end_index]

In [ ]:
def syl2str(a_list, delim='/'): return ' '.join(a_list).replace('++ --', delim)
print(syl2str(random_chunk()))

## GPU

In [ ]:
import torch

USE_GPU = torch.cuda.is_available(); 
# USE_GPU = False; 

print(f'USE_GPU={USE_GPU}')

def to_gpu(x, *args, **kwargs):
    return x.cuda(*args, **kwargs) if USE_GPU else x

## Budowanie modelu

Ten model przyjmie jako wejściie token dla kroku $ t _ {- 1} $ i ma wyprowadzić następny token $ t $. Istnieją trzy warstwy - jedna warstwa liniowa, która koduje znak wejściowy do stanu wewnętrznego, jedna warstwa GRU (która może sama mieć wiele warstw), która działa na tym stanie wewnętrznym i stanie ukrytym, oraz warstwa dekodera, która wyprowadza rozkład prawdopodobieństwa.

In [ ]:
import torch
import torch.nn as nn
from torch.autograd import Variable

class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, n_layers=1):
        super(RNN, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        
        self.encoder = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers)
        self.decoder = nn.Linear(hidden_size, output_size)
    
    def forward(self, input, hidden):
        input = self.encoder(input.view(1, -1))
        output, hidden = self.gru(input.view(1, 1, -1), hidden)
        output = self.decoder(output.view(1, -1))
        return output, hidden

    def init_hidden(self):
        return Variable(to_gpu(torch.zeros(self.n_layers, 1, self.hidden_size)))

## Tensory wejściowe i docelowe

Każdy 'kawałek' zostanie przekształcony w tensor, a dokładnie w `LongTensor` (używany do wartości całkowitych), poprzez przepuszczenie wszystkich tokenów ciągu i wyszukiwanie indeksu każdej sylaby w `all_tokens`.

In [42]:
# Turn token list into list of longs
def tok_tensor(token_list):
    tensor = torch.zeros(len(token_list)).long()
    for c in range(len(token_list)):
        tensor[c] = all_tokens.index(token_list[c])
    return Variable(to_gpu(tensor))

In [ ]:
a_token_list = file_tok[20:30]; print(a_token_list)
print(tok_tensor(a_token_list))

Wreszcie możemy zmontować parę tensorów wejściowych i docelowych do treningu, z losowego kawałka. Wejściem zostaną wszystkie tokeny * aż do przedostatniego*, a celem (targetem) będą wszystkie tokeny * od drugiego*. Jeśli więc nasz kawałek to "abc", wejście będzie odpowiadać "ab", podczas gdy cel to "bc".

In [ ]:
def random_training_set():    
    chunk = random_chunk()
    inp = tok_tensor(chunk[:-1])
    target = tok_tensor(chunk[1:])
    return inp, target

## Ewaluacja

Aby ocenić sieć, będziemy podawać po jednym tokenie na raz, wykorzystywać wyjścia sieci jako rozkład prawdopodobieństwa dla następnego znaku i powtarzać. Aby rozpocząć generowanie, przekazujemy ciąg wstępny, aby rozpocząć budowanie stanu ukrytego, z którego następnie generujemy po jednym tokenie na raz.

In [ ]:
def evaluate(prime_tokl=[all_tokens[1]], predict_len=100, temperature=0.8):
    hidden = decoder.init_hidden()
    prime_input = tok_tensor(prime_tokl)
    predicted = list(prime_tokl)  # need a copy of the list

    # Use priming token list to "build up" hidden state
    for p in range(len(prime_tokl) - 1):
        _, hidden = decoder(prime_input[p], hidden)
    inp = prime_input[-1]
    
    for p in range(predict_len):
        output, hidden = decoder(inp, hidden)
        
        # Sample from the network as a multinomial distribution
        output_dist = output.data.view(-1).div(temperature).exp()
        top_i = torch.multinomial(output_dist, 1)[0]
        
        # Add predicted character to token list and use as next input
        predicted_token = all_tokens[top_i]
        predicted.append(predicted_token)
        inp = tok_tensor([predicted_token])

    return predicted

## Trenowanie modelu

Funkcja pomocnicza do wydrukowania upływającego czasu:

In [ ]:
import time, math

def time_since(since):
    s = time.time() - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

Główna funkcja treningowa

In [ ]:
def train(inp, target):
    hidden = decoder.init_hidden()
    decoder.zero_grad()
    loss = 0

    for c in range(chunk_len):
        output, hidden = decoder(inp[c], hidden)
        loss += criterion(output, target[c].expand(1))

    loss.backward()
    decoder_optimizer.step()

    return loss.item() / chunk_len

Opcjonalny monitoring postępu treningu

In [ ]:
USE_VISDOM = False

import numpy as np

vis = None
if USE_VISDOM:
    import visdom
    vis = visdom.Visdom(port=8890)

def vis_update_line_chart(vis, name, x, y, first_step):
    if not USE_VISDOM: return
    vis.line(Y=np.array([y]), X=np.array([x]), win=name, opts=dict(title=name),
             update=None if first_step else 'append')

def vis_update_text_win(vis, name, text):
    if not USE_VISDOM: return
    vis.text(text, win=name, opts=dict(title=name), append=False)

In [ ]:
class X(str):
    def rpl(self, p, c='lightgray'):
        return X(self.replace(p, f'<font color="{c}">{p}</font>'))

In [ ]:
def bad_words(e_syl): e_str = syl2str(e_syl); return (e_str.count('++') + e_str.count('--')) / len(e_syl)

Następnie definiujemy parametry treningowe i rozpoczynamy trening:

In [ ]:
n_epochs = 2000   # 2000
print_every = 100 # 100
plot_every = 50
hidden_size = 300 # 100
n_layers = 2 # 1
lr = 0.005

decoder = RNN(n_tokens, hidden_size, n_tokens, n_layers)
if USE_GPU:
    decoder.cuda()
print(decoder, flush=True)

decoder_optimizer = torch.optim.Adam(decoder.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()
if USE_GPU:
    criterion.cuda()

start = time.time()
all_losses = []
loss_avg = 0
all_bw = []
bw_avg = 0

from tqdm import tqdm

iterable = range(1, n_epochs + 1)
tqdm_ = tqdm(iterable, '', leave=False, dynamic_ncols=True)
first_step = True

for epoch in tqdm_:
    loss = train(*random_training_set())       
    loss_avg += loss

    # current loss chart
    vis_update_line_chart(vis, 'loss', epoch, loss, epoch == 1)
    
    # progress_bar
    tqdm_.set_postfix({'loss': loss})
    text = f'&nbsp;<font color="red">{tqdm_}</font>'
    vis_update_text_win(vis, 'progress_bar', text)

    # bad words
    prime_str = file_tok[13:18]
    bw = bad_words(evaluate(prime_str, 100))
    bw_avg += bw

    # current bad words chart
    vis_update_line_chart(vis, 'bad_words', epoch, bw, epoch == 1)
    
    if epoch % print_every == 0:
        e_syl = evaluate(prime_str, 100)
        e_str = syl2str(e_syl)
        stats_str = '\n[%s (%d %d%%) loss=%.4f bw=%.4f]' % (time_since(start), epoch, epoch / n_epochs * 100, loss, bw)
        print(stats_str)
        e_html = X(e_str).rpl('/').rpl('--', c='red').rpl('++', c='red')
        print(e_str, '\n', flush=True)
        text = f'<b>{stats_str}</b><br />{e_html}'
        vis_update_text_win(vis, 'evaluation', text)

    if epoch % plot_every == 0:
        vis_update_line_chart(vis, 'loss_avg', epoch, loss_avg / plot_every, first_step)
        vis_update_line_chart(vis, 'bad_words_avg', epoch, bw_avg / plot_every, first_step)
        all_bw.append(bw)
        bw_avg = 0
        first_step = False
        all_losses.append(loss_avg / plot_every)
        loss_avg = 0

## Rysowanie wykresu funkcji straty

Wykreślanie historycznej straty z all_losses pokazuje uczenie sieci:

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
%matplotlib inline

import matplotlib as mpl
mpl.style.use('default')
mpl.style.use('bmh')

plt.figure()
plt.plot(all_losses)

## Zapisanie modelu

In [ ]:
ALLTOKS, MODEL = ['all_tokens', 'model']

fn_pan_tadeusz = {ALLTOKS: 'all_tokens.pan_tadeusz.p', MODEL: 'pan_tadeusz.h300.l2.e3000.gpu.torch'}

fn_dict = fn_pan_tadeusz

In [ ]:
# save all_tokens
import pickle

pickle.dump(all_tokens, open(tmp_path / fn_dict[ALLTOKS], 'wb'))

In [ ]:
# all_tokens = pickle.load( open( tmp_path / fn_dict[ALLTOKS], 'rb' ) )
# n_characters = len(all_tokens)

In [ ]:
decoder.state_dict

In [ ]:
import warnings
warnings.filterwarnings('ignore')

# save model
model_path = tmp_path / fn_dict[MODEL]
torch.save(decoder, model_path)

In [ ]:
 # decoder = torch.load(model_path)

In [ ]:
model_path

In [ ]:
ls -lah $tmp_path

## Ewaluacja w różnych "temperaturach"

W powyższej funkcji `evaluate`, za każdym razem, gdy dokonywana jest prognoza, wyjścia są dzielone przez przekazany argument "temperature". Użycie większej liczby sprawia, że wszystkie akcje są bardziej jednakowo prawdopodobne, a tym samym dają nam "bardziej losowe" wyniki. Użycie mniejszej wartości (mniejszej niż 1) sprawia, że wysokie prawdopodobieństwa przyczyniają się bardziej. Gdy ustawiamy temperaturę na zero, wybieramy tylko najbardziej prawdopodobne wyjścia.

Możemy zobaczyć te efekty poprzez dostosowanie argumentu `temperature`.


In [ ]:
prime_tokl = file_tok[13:18]
print(syl2str(evaluate(prime_tokl, 200, temperature=0.8), delim='/'))

Niższe temperatury daja mniejszą różnorodność, wybierając tylko bardziej prawdopodobne wyjścia:

In [ ]:
print(syl2str(evaluate(prime_tokl, 200, temperature=0.2), delim='/'))

Wyższe temperatury są bardziej różnorodne, wybierając mniej prawdopodobne wyjścia:

In [ ]:
print(syl2str(evaluate(prime_tokl, 200, temperature=1.4), delim='/'))

## Monitorowanie maszyny wirtualnej

In [ ]:
import os
import psutil

def print_memsize():
  process = psutil.Process(os.getpid())
  print(f'{process.memory_info().rss / 1024**3:.5} GB')

In [ ]:
print_memsize()

In [ ]:
!uptime

In [ ]:
# who

## Ćwiczenia

* Trenuj z własnym zestawem danych, np.
     * Tekst od innego autora
     * Posty na blogu
     * Kody źródłowe
* Zwiększ liczbę warstw i rozmiar sieci, aby uzyskać lepsze wyniki

**Następnie**: [Generating Names with a Conditional Character-Level RNN](https://github.com/spro/practical-pytorch/blob/master/conditional-char-rnn/conditional-char-rnn.ipynb)